In [1]:
import argparse
import os
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.utils.data as data
from PIL import Image
from tqdm import tqdm 
# from loss.SoftTriple import SoftTriple
from loss.softtriple import SoftTriple
import evaluation as eva
# import net
from net.bninception import bninception
# from net.resnet_model import ResNet50
# from net.resnet_model_cls import ResNet50
# from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [2]:
datasettttt = '/root/notebooks/nfs/work/jason.chen/project/triplet_loss/pytorch-triplet-loss/CUB_2'


traindir = os.path.join(datasettttt, 'train')
testdir = os.path.join(datasettttt, 'test')

In [3]:
def RGB2BGR(im):
    assert im.mode == 'RGB'
    r, g, b = im.split()
    return Image.merge('RGB', (b, g, r))


In [4]:
normalize = transforms.Normalize(mean=[104., 117., 128.],
                                 std=[1., 1., 1.])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.Lambda(RGB2BGR),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255)),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, shuffle=True,
    num_workers=1, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Lambda(RGB2BGR),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255)),
        normalize,
    ])),
    batch_size=16, shuffle=False,
    num_workers=1, pin_memory=True)

In [5]:
model = bninception(512)
#     model = ResNet50(num_classes=args.dim)
torch.cuda.set_device(1)
model = model.cuda()

In [6]:
criterion = SoftTriple().cuda()
optimizer = torch.optim.Adam([{"params": model.parameters(), "lr": 0.0001},
                              {"params": criterion.parameters(), "lr": 0.01}],
                             eps=0.01, weight_decay=1e-4)

In [7]:
img, y = next(iter(train_loader))

In [8]:
y

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [9]:
img.shape

torch.Size([16, 3, 224, 224])

In [10]:
gpu = 1

In [11]:
img, y = map(lambda x: x.to(gpu), [img , y])

In [12]:
output = model(img)

In [13]:
output.shape

torch.Size([16, 512])

In [14]:
loss, preds = criterion(output, y)

In [15]:
y.shape

torch.Size([16])